In [113]:
import os
import random
from io import open
import unicodedata
import string
import re

import torch
import torchaudio
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torch import optim
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from pathlib import Path
import kaldi_io
import math
import torch.utils.data as Data

from lib.Data_show import Data_show
from lib.Phone_cla_Dataset import Phone_cla_Dataset
from lib.Decoder import Decoder
from collections import Counter

%matplotlib inline


In [59]:
os.environ["CUDA_VISIBLE_DEVICES"]="3"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## 加载数据SLR85训练集

In [2]:
SLR_feats = { u:d for u,d in kaldi_io.read_mat_scp("/home1/meichaoyang/Dataset/feats/SLR85/hifi/train/feats.scp") }
SLR_feats_dev = { u:d for u,d in kaldi_io.read_mat_scp("/home1/meichaoyang/Dataset/feats/SLR85/hifi/dev/feats.scp") }

In [ ]:
phone_label = { u:d for u,d in kaldi_io.read_vec_int_ark("../../wake_up_align_44_1k/train_fbank/ali.1.ph") }
feats = { u:d for u,d in kaldi_io.read_mat_scp("../../wake_up_align_44_1k/train_fbank/feats.scp") }

In [ ]:
# feats_mid_300k = { u:d for u,d in kaldi_io.read_mat_scp("../../wake_up_align_44_1k/train_fbank/feats_mid_300000.scp") }
feats_tail_300k = { u:d for u,d in kaldi_io.read_mat_scp("../../wake_up_align_44_1k/train_fbank/feats_tail_300000.scp") }

In [10]:
aishell_2_corp_raw = "/home1/meichaoyang/Dataset/data_aishell2/aishell2/aishell2/data/trans.txt"
utt_with_you = []
with open(aishell_2_corp_raw, "r") as f:
    for line in f:
        data = line.split()
        if "你" not in data[1]: ##删除小于10和非英文标注
            continue
        utt_with_you.append(data[0])

In [29]:
i

78177

78177

In [30]:
feats_mid_300k = {}
i=0
for u,d in kaldi_io.read_mat_scp("../../wake_up_align_44_1k/train_fbank/feats_mid_300000.scp"):
    if i > 1000000:
        break
    i += 1
    feats_mid_300k[u] = d

In [31]:
feats = {**feats, **feats_mid_300k}

In [32]:
len(feats)

895985

895985

In [116]:
i=0
for utt in utt_with_you:
    if utt in phone_label.keys():
        i += 1
i

2819

2819

In [37]:
len(utt_with_you)

27765

27765

In [ ]:
feats_with_you = {}
for utt in utt_with_you:
    if utt in feats.keyseys():
        feats_with_you[utt]=feats[utt]

In [38]:
feats_aishell = {}
a = random.sample(feats.keys(), 20000)
for utt in a:
    feats_aishell[utt] = feats[utt]
    

In [39]:
len(feats_aishell)

20000

20000

In [40]:
feat_new = {**feats_with_you, **SLR_feats, **feats_aishell }

In [42]:
phone_label_new = {}
for utt in feat_new.keys():
    if utt in phone_label.keys():
        phone_label_new[utt] = phone_label[utt]

In [43]:
len(feat_new)

48806

48806

In [44]:
len(phone_label_new)

6770

6770

In [46]:
len(phone_label)

91706

91706

In [ ]:
phone_label_new.keys()

In [82]:
new_key1 = list(aishell_feats.keys())[0:8000]
new_key2 = list(SLR_feats.keys())
new_key = new_key1 + new_key2
phone_label_new={}
feats_new = {}
for key in new_key:
    phone_label_new[key] = phone_label[key]
    feats_new[key] = feats[key]

KeyError: 'IC0001W0017'

In [95]:
len(aishell_feats)

50001

In [59]:
new_feat = {**feat_new, **SLR_feats}

In [147]:
print(len(aishell_feats))
print(len(phone_label))

100001
91706


## 自定义数据集

In [54]:
Phone_cla_Dataset.maxClassNum = 9

In [48]:
Data_show().phone2class

{1: 0, 3: 0, 129: 1, 63: 2, 61: 3, 27: 4, 128: 5, 64: 6, 92: 7, 69: 8}

{1: 0, 3: 0, 129: 1, 63: 2, 61: 3, 27: 4, 128: 5, 64: 6, 92: 7, 69: 8}

In [55]:
data_set_train = Phone_cla_Dataset(phone_label_new, feat_new)

In [145]:
data_set_dev = Phone_cla_Dataset(phone_label, SLR_feats_dev)

In [146]:
print(data_set_dev.feats_nd.shape)
print(data_set_dev.phone_label_nd.shape)
print(data_set_dev.phone_label_nd[1440])

(47751, 40)
(47751,)
4


In [57]:
data_set_train.phone_label_nd[-75:-61]

array([9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9])

array([9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9])

In [58]:
print(data_set_train.feats_nd.shape)
print(data_set_train.phone_label_nd.shape)
print(data_set_train.phone_label_nd[1440])

(1791993, 40)
(1791993,)
9
(1791993, 40)
(1791993,)
9


In [147]:
train_data = torch.Tensor(data_set_train.feats_nd).to(device)
train_label = torch.LongTensor(data_set_train.phone_label_nd).to(device)
test_data = torch.Tensor(data_set_dev.feats_nd).to(device)
test_label = torch.LongTensor(data_set_dev.phone_label_nd).to(device)

In [114]:
print('Counter(data)\n',Counter(data_set_train.phone_label_nd))

Counter(data)
 Counter({9: 1000645, 0: 442259, 6: 63741, 8: 50976, 1: 46356, 4: 44691, 7: 43785, 3: 43032, 2: 29856, 5: 26652})
Counter(data)
 Counter({9: 1000645, 0: 442259, 6: 63741, 8: 50976, 1: 46356, 4: 44691, 7: 43785, 3: 43032, 2: 29856, 5: 26652})


# 模型搭建

In [61]:
class DNN(nn.Module):
    def __init__(self, input_size, num_classes):
        super().__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 128)
        self.fc4 = nn.Linear(128, 128)
        self.fc5 = nn.Linear(128, num_classes)


    def forward(self, input):
        x = torch.sigmoid(self.fc1(input))
        x = torch.sigmoid(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        x = torch.sigmoid(self.fc4(x))
        x = self.fc5(x)
        
        return x

## 训练迭代

In [164]:
LEARNING_RATE = 0.001  #0.001
EPOCH = 2      #400 best
BATCH_SIZE = 150
input_size=40
num_classes=10

In [165]:
print(train_data.shape)
print(train_label.shape)

torch.Size([1791993, 40])
torch.Size([1791993])


In [166]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# prepare the data loader

training_set = Data.TensorDataset(train_data,
                                  train_label)
training_loader = Data.DataLoader(dataset=training_set,
                                      batch_size=BATCH_SIZE,
                                      shuffle=True)
testing_set = Data.TensorDataset(test_data,
                                 test_label)
testing_loader = Data.DataLoader(dataset=testing_set,
                                     batch_size=BATCH_SIZE,
                                     shuffle=False)
model = DNN(input_size, num_classes).to(device)
criterion = nn.CrossEntropyLoss(weight=torch.tensor([3.5,3.0,3.0,2.5,2.5,5,2.0,2.5,1.5,1.0]).to(device))
optim = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
for epoch in range(EPOCH):
    correct_train = 0
    total_train = 0
    
    total_loss = 0
    ite = 0
    print_every = 2000
    for (data, label) in training_loader:
        ite +=1
        data = data
        label = label
        pred_label = model(data)
        loss = criterion(pred_label, label)
        optim.zero_grad()
        loss.backward()
        total_loss += loss.item()
        optim.step()
        _, answer = torch.max(pred_label.data, 1)
        total_train += label.size(0)
        correct_train += (answer == label).sum()
        if ite % print_every == 0:
            print("total_loss:",total_loss/print_every, "\tloss:",loss.item())
            total_loss=0
    print('Epoch {:3d} Accuracy on training data: {}% ({}/{})'
          .format(epoch, (100 * correct_train / total_train), correct_train, total_train))
    # pytorch 0.4 feature, not calculate grad on test set
    with torch.no_grad():
        correct_test = 0
        total_test = 0
        for (data, label) in testing_loader:
            pred_label = model(data)
            _, answer = torch.max(pred_label.data, 1)
            total_test += label.size(0)
            correct_test += (answer == label).sum()
        print('          Accuracy on testing data: {}% ({}/{})'
              .format((100 * correct_test / total_test), correct_test, total_test))


total_loss: 1.1077494562566281 	loss: 0.9154451489448547
total_loss: 0.8521879502236843 	loss: 0.7632349729537964
total_loss: 0.7981932031810284 	loss: 0.6673657894134521
total_loss: 0.7761694599837065 	loss: 0.6174341440200806
total_loss: 0.7609841782003641 	loss: 0.7500817775726318
Epoch   0 Accuracy on training data: 76% (1365184/1791993)
          Accuracy on testing data: 50% (23966/47751)
total_loss: 0.7408076551556587 	loss: 0.8927361369132996
total_loss: 0.7189966217130422 	loss: 0.5658644437789917
total_loss: 0.7134265313148499 	loss: 0.5394191741943359
total_loss: 0.7065863145291805 	loss: 0.8600566387176514
total_loss: 0.6969027117788792 	loss: 0.618781328201294
Epoch   1 Accuracy on training data: 79% (1423860/1791993)
          Accuracy on testing data: 52% (25086/47751)


### 预测

In [182]:
# model = torch.load('model.pkl')
# model.to(device)
test_feats["miya_mcy"][0:20]

array([[12.489892 , 12.28557  , 11.494011 , 10.989588 , 10.325792 ,
         9.214449 ,  8.981844 ,  9.74506  ,  9.609121 ,  9.6747055,
         9.200644 ,  9.204725 , 10.612509 , 11.347557 , 11.499212 ,
        11.406498 , 10.541653 , 12.271705 , 12.035969 , 12.748648 ,
        12.208368 , 11.035785 , 11.252329 ,  9.832628 , 10.349963 ,
        10.627583 ,  9.921973 , 10.455713 , 10.383152 , 10.018633 ,
         9.718319 , 10.283143 , 10.805543 , 10.501538 , 10.390841 ,
        10.288891 ,  9.796752 , 10.491543 , 10.520856 , 10.878244 ],
       [10.927893 , 11.439659 ,  9.652666 ,  8.719799 , 10.325792 ,
         9.256556 ,  8.909582 , 10.415248 , 10.570845 , 10.534166 ,
         8.348411 ,  9.3243265,  9.249571 , 10.921649 , 10.897419 ,
        10.512176 ,  9.926722 , 10.566521 , 11.2953005, 12.135638 ,
        11.559294 , 11.317964 , 10.746571 , 10.034743 , 10.538596 ,
        10.908701 ,  9.989633 , 10.374117 , 10.307484 , 10.445533 ,
        10.321822 ,  9.803214 , 10.172053 , 10.

In [183]:
torch.Tensor(test_feats["miya_mcy"][0:20].reshape(-1))

tensor([12.4899, 12.2856, 11.4940, 10.9896, 10.3258,  9.2144,  8.9818,  9.7451,
         9.6091,  9.6747,  9.2006,  9.2047, 10.6125, 11.3476, 11.4992, 11.4065,
        10.5417, 12.2717, 12.0360, 12.7486, 12.2084, 11.0358, 11.2523,  9.8326,
        10.3500, 10.6276,  9.9220, 10.4557, 10.3832, 10.0186,  9.7183, 10.2831,
        10.8055, 10.5015, 10.3908, 10.2889,  9.7968, 10.4915, 10.5209, 10.8782,
        10.9279, 11.4397,  9.6527,  8.7198, 10.3258,  9.2566,  8.9096, 10.4152,
        10.5708, 10.5342,  8.3484,  9.3243,  9.2496, 10.9216, 10.8974, 10.5122,
         9.9267, 10.5665, 11.2953, 12.1356, 11.5593, 11.3180, 10.7466, 10.0347,
        10.5386, 10.9087,  9.9896, 10.3741, 10.3075, 10.4455, 10.3218,  9.8032,
        10.1721, 10.3082, 10.2431, 10.5622,  9.9259,  9.8946, 10.4521, 10.8039,
        12.2315, 11.4945, 11.2638, 10.7507,  7.5042,  8.5829,  9.8129,  9.4636,
        10.3960, 10.2656,  9.2662,  9.3841, 10.0673, 10.4766, 11.5313, 11.0474,
        10.3084, 10.6649, 10.5586, 11.81

In [168]:
# test_feats = { u:d for u,d in kaldi_io.read_mat_scp("/home1/meichaoyang/workspace/align_44_1k/test_feat/feats.scp")}

pred_label = model(torch.Tensor(test_feats["miya_mcy"]).to(device))
_, answer = torch.max(pred_label.data, 1)
answer_list=list(answer.to("cpu", torch.int).numpy())

In [173]:
# utt='SV0255_2_00_F0021'
utt="SV0255_7_01_S3881"
# utt=list(phone_label.keys())[110]

# label_list = list(phone_label_dev[utt])
pred_label = model(torch.Tensor(SLR_feats_dev[utt]).to(device))
_, answer = torch.max(pred_label.data, 1)
answer_list=list(answer.to("cpu", torch.int).numpy())

In [ ]:
utt=list(feats_aishell.keys())[11]

label_list = list(feats_aishell[utt])
pred_label = model(torch.Tensor(feats_aishell[utt]).to(device))
_, answer = torch.max(pred_label.data, 1)
answer_list=list(answer.to("cpu", torch.int).numpy())

## 将预测结果映射

In [174]:
decoder = Decoder(Data_show.phone2class)

In [175]:
title1, content1 = decoder.show_result(decoder.decode(torch.nn.Softmax()(pred_label)))

/home1/meichaoyang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  """Entry point for launching an IPython kernel.


In [176]:
title, content = Data_show().show_softmax(torch.nn.Softmax()(pred_label))

/home1/meichaoyang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  """Entry point for launching an IPython kernel.


In [177]:
print(title,content)

	sil	nsn	n	i2	h	ao3	m	i3	ii	ia3	other
 0:	0.84	0.03	0.00	0.00	0.00	0.01	0.01	0.00	0.00	0.11	
1:	0.90	0.03	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.06	
2:	0.94	0.02	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.04	
3:	0.95	0.01	0.00	0.00	0.00	0.00	0.00	0.00	0.00	0.03	
4:	0.76	0.07	0.00	0.00	0.00	0.01	0.01	0.00	0.00	0.13	
5:	0.16	0.09	0.00	0.00	0.00	0.74	0.00	0.00	0.00	0.00	
6:	0.34	0.19	0.00	0.00	0.00	0.43	0.01	0.00	0.00	0.03	
7:	0.47	0.09	0.00	0.00	0.00	0.43	0.00	0.00	0.00	0.00	
8:	0.13	0.08	0.00	0.00	0.00	0.77	0.00	0.00	0.00	0.01	
9:	0.11	0.14	0.00	0.00	0.00	0.75	0.01	0.00	0.00	0.00	
10:	0.01	0.13	0.41	0.00	0.00	0.10	0.10	0.24	0.01	0.00	
11:	0.01	0.13	0.39	0.00	0.00	0.09	0.12	0.26	0.00	0.00	
12:	0.00	0.14	0.37	0.00	0.00	0.09	0.14	0.25	0.00	0.00	
13:	0.00	0.11	0.39	0.00	0.00	0.06	0.15	0.29	0.00	0.00	
14:	0.00	0.09	0.41	0.00	0.00	0.05	0.14	0.30	0.00	0.00	
15:	0.00	0.07	0.43	0.00	0.00	0.03	0.13	0.33	0.00	0.00	
16:	0.00	0.06	0.43	0.00	0.00	0.02	0.12	0.35	0.00	0.00	
17:	0.00	0.08	0.42	0.00	0.00	0.03	0.

In [96]:
utt

'IC0104W0428'

'IC0104W0428'

### “你好米雅”测试

In [ ]:
feats_miya_test = { u:d for u,d in kaldi_io.read_mat_scp("/home1/meichaoyang/Dataset/feats/SLR85/far_field/train/feats.scp") }

In [ ]:
utt_miya_test=list(feats_miya_test.keys())[420]
# utt_aishell="IC0001W0406"

pred_label_miya_test = model(torch.Tensor(feats_miya_test[utt_miya_test]).to(device))
_, answer_miya_test = torch.max(pred_label_miya_test.data, 1)
answer_miya_test_list=list(answer_miya_test.to("cpu", torch.int).numpy())

In [ ]:
title, content = Data_show().show_softmax(torch.nn.Softmax()(pred_label_miya_test))

In [ ]:
print(title)
print(content)

In [ ]:
feats[utt_miya_test]

## 非“你好米雅”测试

In [ ]:
feats_aishell = { u:d for u,d in kaldi_io.read_mat_scp("../wake_dnn_miya_only/feats_aishell2_test/feats.scp") }

In [ ]:
utt_aishell=list(feats_aishell.keys())[420]
utt_aishell="IC0001W0406"

pred_label_aishell = model(torch.Tensor(feats_aishell[utt_aishell]).to(device))
_, answer_aishell = torch.max(pred_label_aishell.data, 1)
answer_aishell_list=list(answer_aishell.to("cpu", torch.int).numpy())

In [ ]:
decoder.show_result(decoder.decode(torch.nn.Softmax()(pred_label_aishell)))

In [ ]:
Data_show().show_softmax(torch.nn.Softmax()(pred_label_aishell))

In [ ]:
utt_aishell

## 保存模型

In [184]:
torch.save(model.to("cpu"), 'model.pkl')
model1 = torch.load('model.pkl')

/home1/meichaoyang/anaconda3/lib/python3.7/site-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type DNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [ ]:
sm = torch.jit.script(model1)
sm.save("phone_cla_model.pt")

In [ ]:
model1 = torch.load('model.pkl')

In [ ]:
model1

In [ ]:
map1={1:"1-1",2:"2-1",3:"3-1"}

In [ ]:
map2={2:"2-2",3:"3-2",4:"4-2"}

In [ ]:
{**map1,**map2}

In [ ]:
{**map2,**map1}